In [1]:
import numpy as np

In [2]:
class Operation():
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
            
    def comput(self):
        pass

In [3]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [4]:
class multiply(Operation):
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [5]:
class matmul(Operation):
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var.dot(y_var)

In [6]:
class Placeholder():
    def __init__(self):
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [7]:
class Variable():
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [8]:
class Graph():
    
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

z = Ax + b

A = 10

b = 1

z = 10x + 1

In [9]:
g = Graph()
g.set_as_default()

In [10]:
A = Variable(10)
b = Variable(1)
x = Placeholder()

In [11]:
y = multiply(A, x)
z = add(y, b)

In [12]:
def traverse_postorder(operation):
    nodes_postorder = []
    
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
    
    recurse(operation)
    return nodes_postorder

In [13]:
for node in traverse_postorder(z):
    if isinstance(node, Variable):
        print(node, node.value)
    else:
        print(node)

<__main__.Variable object at 0x109306048> 10
<__main__.Variable object at 0x1093066a0> 1


In [14]:
class Session():
    
    def run(self, operation, feed_dict={}):
        for node in traverse_postorder(operation):
            if isinstance(node, Placeholder):
                node.output = feed_dict[node]
            elif isinstance(node, Variable):
                node.output = node.value
            else: #Operation
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)

            if type(node.output) == list:
                node.output = np.array(node.output)
        return operation.output

In [15]:
sess = Session()

In [16]:
result = sess.run(operation=z, feed_dict={x:10})

In [17]:
result

101

In [18]:
z.output

101

In [19]:
A.output

10

In [23]:
A.output_nodes

In [20]:
b.output

1

In [21]:
x.output

10

In [22]:
y.output

100